In [20]:
from openai import OpenAI 
import json 
import os  
from dotenv import load_dotenv, find_dotenv 
import requests


_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [21]:
# Loading FMP key
FMP_API_KEY: str = os.environ.get("FMP_API_KEY")

# All the functions
def get_income_statement(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_balance_sheet(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_cash_flow_statement(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period=annual&apikey=ETZqiw9M5ObXGLxKylHzoI5Ec70NQFue"
    response = requests.get(url)
    return json.dumps(response.json())

def get_key_metrics(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_ratios(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/ratios-ttm/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_growth(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

In [22]:
available_functions = {
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_cash_flow_statement": get_cash_flow_statement,
    "get_key_metrics": get_key_metrics,
    "get_financial_ratios": get_financial_ratios,  
    "get_financial_growth": get_financial_growth   
}

In [23]:
import time

def run_assistant(user_message):
  assistant = client.beta.assistants.create(
        name="Financial Analyst",
        instructions="Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends.",
        
        tools=[
 
    {
        "type": "function",
        "function": {"name": "get_income_statement",
         "parameters": {
            "type": "object",
                "properties": {  "ticker": {"type": "string"}  },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": { "name": "get_balance_sheet",
            "parameters": {
                "type": "object", 
                "properties": { "ticker": {"type": "string"} },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": { "name": "get_cash_flow_statement",
            "parameters": {
                "type": "object",
                "properties": { "ticker": {"type": "string"} },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": { "name": "get_key_metrics",
            "parameters": {
                "type": "object",
                "properties": { "ticker": {"type": "string"}  },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": {  "name": "get_financial_ratios",
            "parameters": {
                "type": "object",
                "properties": {  "ticker": {"type": "string"} },
                "required": ["ticker"],
            }
        }
    },
    {
        "type": "function",
        "function": { "name": "get_financial_growth",
            "parameters": {
                "type": "object",
                "properties": {  "ticker": {"type": "string"} },
                "required": ["ticker"],
            }
        }
    }
],
        model="gpt-3.5-turbo-1106",
    )


  thread = client.beta.threads.create()


  client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# Start a run
  run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
 
# Monitor and manage the run
  while True:

        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        # Add run steps retrieval here
        run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
        print("Run Steps:", run_steps)

        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    output = function_to_call(**function_args)
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output,
                    })

            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

        elif run.status == "completed":
            # List the messages to get the response
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            for message in messages.data:
                role_label = "User" if message.role == "user" else "Assistant"
                message_content = message.content[0].text.value
                print(f"{role_label}: {message_content}\n")
            break  # Exit the loop after processing the completed run

        elif run.status == "failed":
            print("Run failed.")
            break

        elif run.status in ["in_progress", "queued"]:
            print(f"Run is {run.status}. Waiting...")
            time.sleep(5)  # Wait for 5 seconds before checking again

        else:
            print(f"Unexpected status: {run.status}")
            break

# Example use case

In [24]:
run_assistant("Share comparison of financial health of Microsoft and Apple over the last four years, focusing on their balance sheets and key financial ratios?")


Run Steps: SyncCursorPage[RunStep](data=[RunStep(id='step_P02hUjlB6G1Y9GLC4JRLHM9P', assistant_id='asst_jpN6CDXDtcMtWUil9t71XMID', cancelled_at=None, completed_at=None, created_at=1703357464, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_hJmFydMGTjzagCqQ3RGLIeCg', status='in_progress', step_details=ToolCallsStepDetails(tool_calls=[], type='tool_calls'), thread_id='thread_3yIawF8uk7HvWQ1C4Z6PtKYu', type='tool_calls', expires_at=1703358063)], object='list', first_id='step_P02hUjlB6G1Y9GLC4JRLHM9P', last_id='step_P02hUjlB6G1Y9GLC4JRLHM9P', has_more=False)
Run is in_progress. Waiting...
Run Steps: SyncCursorPage[RunStep](data=[RunStep(id='step_P02hUjlB6G1Y9GLC4JRLHM9P', assistant_id='asst_jpN6CDXDtcMtWUil9t71XMID', cancelled_at=None, completed_at=None, created_at=1703357464, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_hJmFydMGTjzagCqQ3RGLIeCg', status='in_progress', step_deta